In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df  = pd.read_csv("Default.csv")

df = df.sample(frac=1, random_state=1)

df

,Unnamed: 0,default,student,balance,income
9953,9954,No,Yes,776.544280,14229.728124
3850,3851,No,No,1014.599104,51438.710199
4962,4963,No,No,681.693576,33327.113035
3886,3887,No,Yes,969.898685,17940.119127
5437,5438,No,Yes,1577.083581,15230.820600
...,...,...,...,...,...
2895,2896,No,Yes,1270.092810,16809.006452
7813,7814,Yes,No,1598.020831,39163.361056
905,906,No,No,1234.476479,31313.374575
5192,5193,No,No,0.000000,29322.631394


Q: Fit a logistic regression model that uses income and balance to predict default.

In [2]:
#A:

df["default"] = df["default"].replace({"Yes": 1, "No": 0})

X = df[["income", "balance"]]
y = df["default"]
X = sm.add_constant(X) 

log_reg = sm.Logit(y, X).fit()

print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 21 Aug 2023   Pseudo R-squ.:                  0.4594
Time:                        20:30:26   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99

Q: Using the validation set approach, estimate the test error of this
model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

ii. Fit a multiple logistic regression model using only the training observations.

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater
than 0.5.

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=2)

train_ratio = len(train_df) / len(df)
print(train_ratio)

0.8


In [4]:
X_train = train_df[["income", "balance"]]
y_train = train_df["default"]

X_train = sm.add_constant(X_train)
log_reg_result = sm.Logit(y_train, X_train).fit()

print(log_reg_result.summary())

Optimization terminated successfully.
         Current function value: 0.078846
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7997
Method:                           MLE   Df Model:                            2
Date:                Mon, 21 Aug 2023   Pseudo R-squ.:                  0.4656
Time:                        20:30:29   Log-Likelihood:                -630.77
converged:                       True   LL-Null:                       -1180.4
Covariance Type:            nonrobust   LLR p-value:                2.064e-239
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5748      0.486    -23.815      0.000     -12.527     -10.622
income      2.028e-05   5.57

In [5]:
X_test = test_df[["income", "balance"]]
y_test = test_df["default"]

X_test = sm.add_constant(X_test)
log_reg_model = sm.Logit(y_test, X_test)
log_reg_result = log_reg_model.fit()

test_pred_probs = log_reg_result.predict(X_test)

test_pred = ["Yes" if prob > 0.5 else "No" for prob in test_pred_probs]


Optimization terminated successfully.
         Current function value: 0.079175
         Iterations 10


In [6]:
df  = pd.read_csv("Default.csv")

df = df.sample(frac=1, random_state=1)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=2)

discrepancy_mean = round((test_df["default"] != test_pred).mean(),4)

print(discrepancy_mean)

0.0245


Q: Repeat the process in (b) three times, using three different splits of the observations into a train

In [7]:
log_def_errors = []

for i in range(2, 6):

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=i)

    log_reg_model = LogisticRegression()
    log_reg_model.fit(train_df[["income", "balance"]], train_df["default"])

    test_pred_probs1 = log_reg_result.predict(X_test)

    test_pred1 = ["Yes" if prob > 0.5 else "No" for prob in test_pred_probs1]
    
    error = np.mean(test_df["default"] != test_pred)
    log_def_errors.append(error)

rounded_errors = [round(error, 4) for error in log_def_errors]
print(rounded_errors)
print(np.mean(rounded_errors))

[0.0245, 0.0465, 0.045, 0.054]
0.0425


A: There is some variation, as we would expect with the validation set approach and cross-validation. The average test error is 0.0425.

Q: Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [8]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=10)

train_df[["default","student"]]  = train_df[["default","student"]].replace({"Yes": 1, "No": 0})
test_df[["default","student"]]  = test_df[["default","student"]].replace({"Yes": 1, "No": 0})

X_train = train_df.drop("default", axis=1)
y_train = train_df["default"]

X_train = sm.add_constant(X_train)

log_reg_model = sm.Logit(y_train, X_train)
log_reg_result = log_reg_model.fit()

print(log_reg_result.summary())

Optimization terminated successfully.
         Current function value: 0.078204
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7995
Method:                           MLE   Df Model:                            4
Date:                Mon, 21 Aug 2023   Pseudo R-squ.:                  0.4623
Time:                        20:30:34   Log-Likelihood:                -625.63
converged:                       True   LL-Null:                       -1163.5
Covariance Type:            nonrobust   LLR p-value:                1.323e-231
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.4364      0.559    -18.666      0.000     -11.532      -9.341
Unnamed: 0   -4.2e-05   2.68

In [9]:
X_test = test_df.drop("default", axis=1)
X_test = sm.add_constant(X_test)

test_pred_probs = log_reg_result.predict(X_test)
test_pred = ['Yes' if prob > 0.5 else 'No' for prob in test_pred_probs]

In [11]:
df  = pd.read_csv("Default.csv")

df = df.sample(frac=1, random_state=1)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=10)

discrepancy_mean = round((test_df["default"] != test_pred).mean(),4)

print(discrepancy_mean)

0.0275


It seems that the test error of this model is smaller than the test error of the smaller model.